# Bipolar vs Refferential Montages

EEG recordings measure brain activity in real time, are non-invasive, and are relatively low-cost. They have excellent timing but do not pinpoint the exact source of the signals in the brain. One downside is that EEG is sensitive to noise and can pick up unwanted signals or artifacts. In a bipolar montage the difference between two neighboring electrodes is measured, which helps highlight local brain activity and reduces common noise. However, this approach might miss activity that is spread across larger brain regions. In a referential montage, each electrode's signal is compared to a common reference, often the average of all electrodes, to capture a broader picture of brain activity. This method can show overall brain patterns but may introduce extra artifacts related to the choice of reference. The choice of montage depends on whether you need detailed local information or a general view of overall brain activity.

In [1]:
!pip install kagglehub

In [2]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("wajahat1064/emotion-recognition-using-eeg-and-computer-games")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\nolan\.cache\kagglehub\datasets\wajahat1064\emotion-recognition-using-eeg-and-computer-games\versions\2


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load and prepare the EEG data
df = pd.read_csv('/Users/nolan/.cache/kagglehub/datasets/wajahat1064/emotion-recognition-using-eeg-and-computer-games/versions/2/S01G1AllChannels.csv')
df = df.dropna(axis=1, how='all')  # Remove columns that are entirely NaN
channels = df.columns.tolist()
print("Available channels:", channels)


Available channels: ['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7', 'P8', 'T7', 'T8']


In [4]:
# global variables and helper functions
fs = 256  # Sampling rate (Hz)

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

In [5]:
def process_and_plot(b):
    with out:
        clear_output(wait=True)
        # Retrieve widget values
        montage_type = montage_dropdown.value.lower()
        low_freq = low_freq_input.value
        high_freq = high_freq_input.value
        print("Montage:", montage_dropdown.value)
        print("Frequency range: {} - {} Hz".format(low_freq, high_freq))
        
        # Dictionary to hold processed signals
        processed_signals = {}
        
        if montage_type.startswith('reffer'):  # covers 'refferential'
            # Referential montage: subtract the average voltage across all channels
            data_matrix = df[channels].values  # shape: (n_samples, n_channels)
            avg_reference = np.mean(data_matrix, axis=1)
            for ch in channels:
                processed_signals[ch] = df[ch].values - avg_reference
        
        elif montage_type.startswith('bipolar'):
            # Bipolar montage using a double banana approach.
            # Define two chains (if the channels are available):
            # Left chain: AF3 → F3 → FC5 → T7 → P7 → O1
            # Right chain: AF4 → F4 → FC6 → T8 → P8 → O2
            left_chain = [ch for ch in ['AF3', 'F3', 'FC5', 'T7', 'P7', 'O1'] if ch in channels]
            right_chain = [ch for ch in ['AF4', 'F4', 'FC6', 'T8', 'P8', 'O2'] if ch in channels]
            for chain in [left_chain, right_chain]:
                for i in range(len(chain) - 1):
                    ch1 = chain[i]
                    ch2 = chain[i + 1]
                    bipolar_name = f"{ch1}-{ch2}"
                    # Subtract the voltage of the electrode behind from the electrode in front
                    processed_signals[bipolar_name] = df[ch1].values - df[ch2].values
        else:
            print("Invalid montage type selected.")
            return
        
        # Apply the band-pass filter to each processed signal
        for key in processed_signals:
            processed_signals[key] = bandpass_filter(processed_signals[key], low_freq, high_freq, fs)
        
        # Compute stats
        aggregate_stats = {}
        for key, signal in processed_signals.items():
            mean_val = np.mean(signal)
            var_val = np.var(signal)
            # Fano factor: variance divided by the absolute mean (with a safeguard for near-zero mean)
            fano = var_val / np.abs(mean_val) if np.abs(mean_val) > 1e-6 else np.nan
            aggregate_stats[key] = {'mean': mean_val, 'variance': var_val, 'fano_factor': fano}
        
        # Plot each processed channel in a grid (dashboard format)
        n_signals = len(processed_signals)
        cols = 3  # Number of columns in the grid
        rows = int(np.ceil(n_signals / cols))
        fig, axes = plt.subplots(rows, cols, figsize=(15, rows*3), squeeze=False)
        axes = axes.flatten()
        
        for i, (key, signal) in enumerate(processed_signals.items()):
            ax = axes[i]
            ax.plot(signal)
            stats = aggregate_stats[key]
            ax.set_title(f"{key}\nMean: {stats['mean']:.2f}, Var: {stats['variance']:.2f}, Fano: {stats['fano_factor']:.2f}")
            ax.set_xlabel("Sample")
            ax.set_ylabel("Voltage (µV)")
        # Turn off unused subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Display stats as df
        stats_df = pd.DataFrame(aggregate_stats).T
        print("Aggregate statistics for each channel:")
        display(stats_df)



In [6]:
# user input
low_freq_input = widgets.FloatText(value=1.0, description='Low Freq:')
high_freq_input = widgets.FloatText(value=10.0, description='High Freq:')
montage_types = ["Bipolar", "Refferential"]
montage_dropdown = widgets.Dropdown(options=montage_types, description='Montage Type:')
process_button = widgets.Button(description="Process Data")
process_button.on_click(process_and_plot)

out = widgets.Output()

display(montage_dropdown, low_freq_input, high_freq_input, process_button, out)

Dropdown(description='Montage Type:', options=('Bipolar', 'Refferential'), value='Bipolar')

FloatText(value=1.0, description='Low Freq:')

FloatText(value=10.0, description='High Freq:')

Button(description='Process Data', style=ButtonStyle())

Output()